In [59]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly 
import plotly.plotly as py     
import plotly.graph_objs as go  
import cufflinks as cf
import pandas as pd
import folium

flights_data = pd.read_csv('flight_data.csv')
flights_data.sample(3)

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,...,AIR_TIME,FLIGHTS,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,Unnamed: 42
348565,1,1,9,6,2016-01-09,DL,1284,10397,1039705,30397,...,119.0,1.0,782.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1728037,4,12,29,4,2016-12-29,WN,2880,11292,1129202,30325,...,106.0,1.0,977.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1343288,4,11,3,4,2016-11-03,AA,2337,13930,1393004,30977,...,116.0,1.0,802.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
date = flights_data["FL_DATE"]
planned_departure_time = flights_data["CRS_DEP_TIME"]
actual_departure_time = flights_data["DEP_TIME"]
delay = flights_data["DEP_DELAY"]
departure_airport = flights_data["ORIGIN"]
departure_city = flights_data["ORIGIN_CITY_NAME"]
destination_airport = flights_data["DEST"]
destination_city = flights_data["DEST_CITY_NAME"]
flights_df = pd.DataFrame({"Date": date,
                           "Departure Time (Planned)": planned_departure_time,
                           "Departure Time (Actual)": actual_departure_time,
                           "Delays (in minutes)": delay, 
                           "Departure Airport": departure_airport,
                           "Departure City": departure_city,
                           "Destination Airport": destination_airport,
                           "Destination City": destination_city})
flights_df

,Date,Delays (in minutes),Departure Airport,Departure City,Departure Time (Actual),Departure Time (Planned),Destination Airport,Destination City
0,2016-01-03,-1.0,DEN,"Denver, CO",1524.0,1525,CID,"Cedar Rapids/Iowa City, IA"
1,2016-01-03,44.0,PBI,"West Palm Beach/Palm Beach, FL",744.0,700,DEN,"Denver, CO"
2,2016-01-03,13.0,TTN,"Trenton, NJ",1858.0,1845,RDU,"Raleigh/Durham, NC"
3,2016-01-03,-1.0,RDU,"Raleigh/Durham, NC",2054.0,2055,TTN,"Trenton, NJ"
4,2016-01-03,2.0,TTN,"Trenton, NJ",1252.0,1250,ORD,"Chicago, IL"
5,2016-01-03,19.0,ORD,"Chicago, IL",1519.0,1500,TTN,"Trenton, NJ"
6,2016-01-03,-4.0,TTN,"Trenton, NJ",556.0,600,MCO,"Orlando, FL"
7,2016-01-03,22.0,MCO,"Orlando, FL",957.0,935,TTN,"Trenton, NJ"
8,2016-01-03,-2.0,ATL,"Atlanta, GA",1743.0,1745,MSY,"New Orleans, LA"
9,2016-01-03,3.0,MSY,"New Orleans, LA",1913.0,1910,DEN,"Denver, CO"


In [3]:
flights_departures_series = flights_data["DEP_DELAY"].value_counts()
flights_departures_series
delays = flights_data[flights_data["DEP_DELAY"] > 0]
delays.count() # 663,054 delays
departure_series = delays["ORIGIN_CITY_NAME"].value_counts() # Chicago has the most delays at 47,049 delays
destination_series = delays["DEST_CITY_NAME"].value_counts() # Atlanta receives the most delays at 39,434 and Chicago in 2nd at 38,121 delays

flight_delays_df = pd.DataFrame({"Number of Departure Delays": departure_series})
flight_delays_df["Location"] = flight_delays_df.index
flight_delays_df

,Number of Departure Delays,Location
"Chicago, IL",47049,"Chicago, IL"
"Atlanta, GA",43494,"Atlanta, GA"
"Los Angeles, CA",33165,"Los Angeles, CA"
"Denver, CO",28894,"Denver, CO"
"Houston, TX",24027,"Houston, TX"
"New York, NY",23826,"New York, NY"
"San Francisco, CA",23750,"San Francisco, CA"
"Dallas/Fort Worth, TX",23068,"Dallas/Fort Worth, TX"
"Las Vegas, NV",20516,"Las Vegas, NV"
"Phoenix, AZ",19498,"Phoenix, AZ"


In [61]:
chicago_delays_df = flights_df[(flights_df["Departure City"] == "Chicago, IL") & (flights_df["Delays (in minutes)"] > 0)]
#chicago_delays_df
chicago_delays_sorted_df = chicago_delays_df.sort_values(["Date", "Departure Time (Actual)"], ascending = True)
chicago_delays_sorted_df

,Date,Delays (in minutes),Departure Airport,Departure City,Departure Time (Actual),Departure Time (Planned),Destination Airport,Destination City
110125,2016-01-01,7.0,ORD,"Chicago, IL",507.0,500,EWR,"Newark, NJ"
110230,2016-01-01,17.0,ORD,"Chicago, IL",521.0,504,IAH,"Houston, TX"
110087,2016-01-01,13.0,ORD,"Chicago, IL",543.0,530,DEN,"Denver, CO"
15703,2016-01-01,15.0,ORD,"Chicago, IL",545.0,530,FLL,"Fort Lauderdale, FL"
222872,2016-01-01,15.0,ORD,"Chicago, IL",545.0,530,DTW,"Detroit, MI"
219436,2016-01-01,13.0,ORD,"Chicago, IL",558.0,545,ATL,"Atlanta, GA"
110078,2016-01-01,6.0,ORD,"Chicago, IL",607.0,601,LAX,"Los Angeles, CA"
108341,2016-01-01,16.0,ORD,"Chicago, IL",709.0,653,SFO,"San Francisco, CA"
406968,2016-01-01,29.0,ORD,"Chicago, IL",727.0,658,RIC,"Richmond, VA"
228115,2016-01-01,1.0,MDW,"Chicago, IL",731.0,730,MCO,"Orlando, FL"


In [63]:
chicago_weather_2016 = pd.read_csv('chicago_weather_2016.csv')
chicago_weather_2016.sample(3)

,Date,Temp high,Temp avg,Temp low,Visibility high,Visibility avg,Visibility low,Wind low,Wind avg,Wind high,Precipitation sum,Events
270,2016-09-11,76,64,51,10,10,10,13,6,17,0,NaN
372,2016-12-16,20,14,7,10,5,0,17,9,21,0.13,"Fog , Snow"
161,NaN,Temp high,Temp avg,Temp low,Visibility high,Visibility avg,Visibility low,Wind low,Wind avg,Wind high,Precipitation sum,Events


In [64]:
chicago_delays_sorted_df

merged_chicago_data = pd.merge(chicago_delays_sorted_df, chicago_weather_2016, how='inner', on=None, left_on="Date", right_on="Date", left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False)
merged_chicago_data

,Date,Delays (in minutes),Departure Airport,Departure City,Departure Time (Actual),Departure Time (Planned),Destination Airport,Destination City,Temp high,Temp avg,Temp low,Visibility high,Visibility avg,Visibility low,Wind low,Wind avg,Wind high,Precipitation sum,Events
0,2016-01-01,7.0,ORD,"Chicago, IL",507.0,500,EWR,"Newark, NJ",31,25,19,10,10,9,22,14,28,T,Snow
1,2016-01-01,17.0,ORD,"Chicago, IL",521.0,504,IAH,"Houston, TX",31,25,19,10,10,9,22,14,28,T,Snow
2,2016-01-01,13.0,ORD,"Chicago, IL",543.0,530,DEN,"Denver, CO",31,25,19,10,10,9,22,14,28,T,Snow
3,2016-01-01,15.0,ORD,"Chicago, IL",545.0,530,FLL,"Fort Lauderdale, FL",31,25,19,10,10,9,22,14,28,T,Snow
4,2016-01-01,15.0,ORD,"Chicago, IL",545.0,530,DTW,"Detroit, MI",31,25,19,10,10,9,22,14,28,T,Snow
5,2016-01-01,13.0,ORD,"Chicago, IL",558.0,545,ATL,"Atlanta, GA",31,25,19,10,10,9,22,14,28,T,Snow
6,2016-01-01,6.0,ORD,"Chicago, IL",607.0,601,LAX,"Los Angeles, CA",31,25,19,10,10,9,22,14,28,T,Snow
7,2016-01-01,16.0,ORD,"Chicago, IL",709.0,653,SFO,"San Francisco, CA",31,25,19,10,10,9,22,14,28,T,Snow
8,2016-01-01,29.0,ORD,"Chicago, IL",727.0,658,RIC,"Richmond, VA",31,25,19,10,10,9,22,14,28,T,Snow
9,2016-01-01,1.0,MDW,"Chicago, IL",731.0,730,MCO,"Orlando, FL",31,25,19,10,10,9,22,14,28,T,Snow


In [65]:
merged_chicago_data["Month"] = np.nan
merged_chicago_data["Month"][merged_chicago_data["Date"] <= "2016-01-31"] = "Jan"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-02-01") & 
                             (merged_chicago_data["Date"] <= "2016-02-31")] = "Feb"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-03-01") & 
                             (merged_chicago_data["Date"] <= "2016-03-31")] = "Mar"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-04-01") & 
                             (merged_chicago_data["Date"] <= "2016-04-31")] = "Apr"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-05-01") & 
                             (merged_chicago_data["Date"] <= "2016-05-31")] = "May"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-06-01") & 
                             (merged_chicago_data["Date"] <= "2016-06-31")] = "June"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-07-01") & 
                             (merged_chicago_data["Date"] <= "2016-07-31")] = "July"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-08-01") & 
                             (merged_chicago_data["Date"] <= "2016-08-31")] = "Aug"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-09-01") & 
                             (merged_chicago_data["Date"] <= "2016-09-31")] = "Sept"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-10-01") & 
                             (merged_chicago_data["Date"] <= "2016-10-31")] = "Oct"
merged_chicago_data["Month"][(merged_chicago_data["Date"] >= "2016-11-01") & 
                             (merged_chicago_data["Date"] <= "2016-11-31")] = "Nov"
merged_chicago_data["Month"][merged_chicago_data["Date"] >= "2016-12-01"] = "Dec"
merged_chicago_data
merged_chicago_data["Month"].value_counts()

Aug    14238
Dec    12806
Jan    10873
Nov     9132
Name: Month, dtype: int64

In [66]:
chicago_events = merged_chicago_data["Events"].value_counts()
chicago_events

Snow                   9052
Rain                   6069
Rain , Thunderstorm    5196
Fog , Snow             1427
Rain , Snow             862
Fog , Rain              722
Fog                     719
Thunderstorm            411
Fog , Rain , Snow       320
Name: Events, dtype: int64

In [67]:
chicago_events_df = pd.DataFrame({"Causes of Delays" : chicago_events})
chicago_events_df

,Causes of Delays
Snow,9052
Rain,6069
"Rain , Thunderstorm",5196
"Fog , Snow",1427
"Rain , Snow",862
"Fog , Rain",722
Fog,719
Thunderstorm,411
"Fog , Rain , Snow",320


In [ ]:
plotly.tools.set_credentials_file(username='kfmunste', api_key='2SsIURhaGEnwTjHZfYtA')

In [91]:
chicago_events_df.iplot(kind = "bar", labels = "Cause of Delays", 
                        values = "Number of Delays", title = "Weather Conditons Causing Delays")

In [75]:
january = merged_chicago_data[merged_chicago_data["Month"] == "Jan"]
august = merged_chicago_data[merged_chicago_data["Month"] == "Aug"]
november = merged_chicago_data[merged_chicago_data["Month"] == "Nov"]
december = merged_chicago_data[merged_chicago_data["Month"] == "Dec"]

In [76]:
january = january["Events"].value_counts()
jan_df = pd.DataFrame({"Weather Events during Delays": january.index, "Counts" : january})
jan_df

,Counts,Weather Events during Delays
Snow,5017,Snow
Rain,1322,Rain
"Rain , Snow",330,"Rain , Snow"
"Fog , Rain , Snow",320,"Fog , Rain , Snow"


In [77]:
august = august["Events"].value_counts()
aug_df = pd.DataFrame({"Weather Events during Delays": august.index, "Counts" : august})
aug_df

,Counts,Weather Events during Delays
"Rain , Thunderstorm",4303,"Rain , Thunderstorm"
Rain,2245,Rain
Thunderstorm,411,Thunderstorm


In [78]:
november = november["Events"].value_counts()
nov_df = pd.DataFrame({"Weather Events during Delays": november.index, "Counts" : november})
nov_df

,Counts,Weather Events during Delays
Rain,2250,Rain
"Rain , Thunderstorm",893,"Rain , Thunderstorm"
Fog,483,Fog
Snow,282,Snow


In [79]:
december = december["Events"].value_counts()
dec_df = pd.DataFrame({"Weather Events during Delays": december.index, "Counts" : december})
dec_df

,Counts,Weather Events during Delays
Snow,3753,Snow
"Fog , Snow",1427,"Fog , Snow"
"Fog , Rain",722,"Fog , Rain"
"Rain , Snow",532,"Rain , Snow"
Rain,252,Rain
Fog,236,Fog


In [83]:
jan_df.iplot(kind="pie", labels = "Weather Events during Delays", values= "Counts", title = "January")

In [84]:
aug_df.iplot(kind="pie", labels = "Weather Events during Delays", values= "Counts", title = "August")

In [87]:
nov_df.iplot(kind="pie", labels = "Weather Events during Delays", values= "Counts", title = "November")

In [88]:
dec_df.iplot(kind="pie", labels = "Weather Events during Delays", values= "Counts", title = "December")

In [90]:
months = merged_chicago_data["Month"].value_counts()
months_df = pd.DataFrame({"Number of Delays" : months})
months_df
months_df.iplot(kind = "bar", labels = "Months", values = "Number of Delays", title = "Number of Delays by Month")

In [94]:
merged_chicago_data

,Date,Delays (in minutes),Departure Airport,Departure City,Departure Time (Actual),Departure Time (Planned),Destination Airport,Destination City,Temp high,Temp avg,Temp low,Visibility high,Visibility avg,Visibility low,Wind low,Wind avg,Wind high,Precipitation sum,Events,Month
0,2016-01-01,7.0,ORD,"Chicago, IL",507.0,500,EWR,"Newark, NJ",31,25,19,10,10,9,22,14,28,T,Snow,Jan
1,2016-01-01,17.0,ORD,"Chicago, IL",521.0,504,IAH,"Houston, TX",31,25,19,10,10,9,22,14,28,T,Snow,Jan
2,2016-01-01,13.0,ORD,"Chicago, IL",543.0,530,DEN,"Denver, CO",31,25,19,10,10,9,22,14,28,T,Snow,Jan
3,2016-01-01,15.0,ORD,"Chicago, IL",545.0,530,FLL,"Fort Lauderdale, FL",31,25,19,10,10,9,22,14,28,T,Snow,Jan
4,2016-01-01,15.0,ORD,"Chicago, IL",545.0,530,DTW,"Detroit, MI",31,25,19,10,10,9,22,14,28,T,Snow,Jan
5,2016-01-01,13.0,ORD,"Chicago, IL",558.0,545,ATL,"Atlanta, GA",31,25,19,10,10,9,22,14,28,T,Snow,Jan
6,2016-01-01,6.0,ORD,"Chicago, IL",607.0,601,LAX,"Los Angeles, CA",31,25,19,10,10,9,22,14,28,T,Snow,Jan
7,2016-01-01,16.0,ORD,"Chicago, IL",709.0,653,SFO,"San Francisco, CA",31,25,19,10,10,9,22,14,28,T,Snow,Jan
8,2016-01-01,29.0,ORD,"Chicago, IL",727.0,658,RIC,"Richmond, VA",31,25,19,10,10,9,22,14,28,T,Snow,Jan
9,2016-01-01,1.0,MDW,"Chicago, IL",731.0,730,MCO,"Orlando, FL",31,25,19,10,10,9,22,14,28,T,Snow,Jan


In [95]:
delay_time_sorted_df = merged_chicago_data.sort_values(["Delays (in minutes)"], ascending = True)
delay_time_sorted_df

,Date,Delays (in minutes),Departure Airport,Departure City,Departure Time (Actual),Departure Time (Planned),Destination Airport,Destination City,Temp high,Temp avg,Temp low,Visibility high,Visibility avg,Visibility low,Wind low,Wind avg,Wind high,Precipitation sum,Events,Month
3294,2016-01-09,1.0,MDW,"Chicago, IL",1321.0,1320,BWI,"Baltimore, MD",40,32,23,8,3,0,26,10,32,0.27,"Fog , Rain , Snow",Jan
22428,2016-08-25,1.0,ORD,"Chicago, IL",1041.0,1040,BOS,"Boston, MA",85,76,67,10,10,6,13,6,16,0.09,"Rain , Thunderstorm",Aug
21601,2016-08-23,1.0,ORD,"Chicago, IL",656.0,655,BHM,"Birmingham, AL",81,71,60,10,10,10,20,12,24,0,NaN,Aug
9095,2016-01-24,1.0,ORD,"Chicago, IL",703.0,702,ROC,"Rochester, NY",34,29,24,10,8,6,20,8,24,0,NaN,Jan
5150,2016-01-13,1.0,ORD,"Chicago, IL",1043.0,1042,DAY,"Dayton, OH",25,12,-2,10,10,8,18,9,23,T,Snow,Jan
9093,2016-01-24,1.0,MDW,"Chicago, IL",601.0,600,FLL,"Fort Lauderdale, FL",34,29,24,10,8,6,20,8,24,0,NaN,Jan
44916,2016-12-26,1.0,ORD,"Chicago, IL",1601.0,1600,LGA,"New York, NY",54,41,28,10,7,0,33,19,44,0.23,"Fog , Rain",Dec
26545,2016-11-05,1.0,MDW,"Chicago, IL",801.0,800,LAS,"Las Vegas, NV",68,56,43,10,10,10,14,7,17,0,NaN,Nov
43543,2016-12-22,1.0,ORD,"Chicago, IL",2101.0,2100,BNA,"Nashville, TN",37,31,24,10,10,9,15,9,19,0,NaN,Dec
7442,2016-01-18,1.0,ORD,"Chicago, IL",2156.0,2155,PHX,"Phoenix, AZ",7,2,-4,10,10,10,20,14,26,0,NaN,Jan


In [96]:
jan_delay_time_sorted_df = delay_time_sorted_df[delay_time_sorted_df["Month"] == "Jan"]
aug_delay_time_sorted_df = delay_time_sorted_df[delay_time_sorted_df["Month"] == "Aug"]
nov_delay_time_sorted_df = delay_time_sorted_df[delay_time_sorted_df["Month"] == "Nov"]
dec_delay_time_sorted_df = delay_time_sorted_df[delay_time_sorted_df["Month"] == "Dec"]

In [97]:
time_of_delays_jan = jan_delay_time_sorted_df["Delays (in minutes)"]
time_of_delays_aug = aug_delay_time_sorted_df["Delays (in minutes)"]
time_of_delays_nov = nov_delay_time_sorted_df["Delays (in minutes)"]
time_of_delays_dec = dec_delay_time_sorted_df["Delays (in minutes)"]

dates_jan = jan_delay_time_sorted_df["Date"]
dates_aug = aug_delay_time_sorted_df["Date"]
dates_nov = nov_delay_time_sorted_df["Date"]
dates_dec = dec_delay_time_sorted_df["Date"]

scatterplot_df = pd.DataFrame({"Date_Jan": dates_jan, 
                               "Date_Aug": dates_aug,
                               "Date_Nov": dates_nov,
                               "Date_Dec": dates_dec,
                               "Delays (in minutes)_jan": time_of_delays_jan,
                               "Delays (in minutes)_aug": time_of_delays_aug,  
                               "Delays (in minutes)_nov": time_of_delays_nov,
                               "Delays (in minutes)_dec": time_of_delays_dec
                              })

scatterplot_df

,Date_Aug,Date_Dec,Date_Jan,Date_Nov,Delays (in minutes)_aug,Delays (in minutes)_dec,Delays (in minutes)_jan,Delays (in minutes)_nov
0,NaN,NaN,2016-01-01,NaN,NaN,NaN,7.0,NaN
1,NaN,NaN,2016-01-01,NaN,NaN,NaN,17.0,NaN
2,NaN,NaN,2016-01-01,NaN,NaN,NaN,13.0,NaN
3,NaN,NaN,2016-01-01,NaN,NaN,NaN,15.0,NaN
4,NaN,NaN,2016-01-01,NaN,NaN,NaN,15.0,NaN
5,NaN,NaN,2016-01-01,NaN,NaN,NaN,13.0,NaN
6,NaN,NaN,2016-01-01,NaN,NaN,NaN,6.0,NaN
7,NaN,NaN,2016-01-01,NaN,NaN,NaN,16.0,NaN
8,NaN,NaN,2016-01-01,NaN,NaN,NaN,29.0,NaN
9,NaN,NaN,2016-01-01,NaN,NaN,NaN,1.0,NaN


In [99]:
plot_data = [go.Scatter(x=scatterplot_df["Date_Jan"], y=scatterplot_df["Delays (in minutes)_jan"], mode="markers", 
               marker= dict(color = "red")), 
]

                               
py.iplot({ 'data': plot_data,
            'layout': {
               'title': 'Chicago Delays in January',
               'xaxis': {
                 'title': 'Date'},
               'yaxis': {
                'title': 'Length of Delay (in minutes)'}
        }})

The draw time for this plot will be slow for clients without much RAM.


In [56]:
plot_data = [go.Scatter(x=scatterplot_df["Date_Aug"], y=scatterplot_df["Delays (in minutes)_aug"], mode="markers", 
               marker= dict(color = "blue")), 
]

                               
py.iplot({ 'data': plot_data,
            'layout': {
               'title': 'Chicago Delays in August',
               'xaxis': {
                 'title': 'Date'},
               'yaxis': {
                'title': 'Length of Delay (in minutes)'}
        }})

The draw time for this plot will be slow for clients without much RAM.


In [57]:
plot_data = [go.Scatter(x=scatterplot_df["Date_Nov"], y=scatterplot_df["Delays (in minutes)_nov"], mode="markers", 
               marker= dict(color = "green")), 
]

                               
py.iplot({ 'data': plot_data,
            'layout': {
               'title': 'Chicago Delays in November',
               'xaxis': {
                 'title': 'Date'},
               'yaxis': {
                'title': 'Length of Delay (in minutes)'}
        }})

The draw time for this plot will be slow for clients without much RAM.


In [58]:
plot_data = [go.Scatter(x=scatterplot_df["Date_Dec"], y=scatterplot_df["Delays (in minutes)_dec"], mode="markers", 
               marker= dict(color = "purple")), 
]

                               
py.iplot({ 'data': plot_data,
            'layout': {
               'title': 'Chicago Delays in December',
               'xaxis': {
                 'title': 'Date'},
               'yaxis': {
                'title': 'Length of Delay (in minutes)'}
        }})

The draw time for this plot will be slow for clients without much RAM.
